In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Dijkstra Algorithm
- brought to you by https://www.redblobgames.com/pathfinding/a-star/introduction.html

In [35]:
from copy import deepcopy
class AmphipodsState:
    def __init__(self,state,coming_from=None, cost_so_far=0):
        self.coming_from = coming_from
        self.cost_so_far = cost_so_far
        self.grid, self.amphi_locs = self.read_state(state)
        
        self.type_cost = {'A': 1,
                          'B': 10,
                          'C': 100,
                          'D': 1000,
                       }
        self.home_entry = {
            'A' : 3,
            'B' : 5,
            'C' : 7,
            'D' : 9
        }
        
    def _create_str(self, grid):
        return '\n'.join([''.join(row) for row in grid])
    
    def __str__(self,):
        return self._create_str(self.grid)
    
    def read_state(self,state):
        lines = state.split('\n')
        grid= []
        grid.append([ i for i in lines[0] ])
        amphi_locs = {'A':[],
                      'B':[],
                      'C':[],
                      'D':[],
                     }
        for r,line in enumerate(lines[1:]):
            new_row = []
            for c,element in enumerate(line):
                if element in  ['#', ' ', '.']:
                    new_row.append(element)
                    continue
                amphi_locs[element].append((r+1, c))
                new_row.append(element)
            grid.append(new_row)
        return grid, amphi_locs
    
    def point_swap(self, cur_grid, pt1, pt2):
        new_grid = deepcopy(cur_grid)
        new_grid[pt1[0]][pt1[1]] = cur_grid[pt2[0]][pt2[1]]
        new_grid[pt2[0]][pt2[1]] = cur_grid[pt1[0]][pt1[1]]
        #print (self._create_str(new_grid))
        return new_grid
                
    def move_amphipods(self,amph, loc):
        home = self.home_entry[amph]
        grid = self.grid.copy()
        cur_cost = self.cost_so_far
        step_cost = self.type_cost[amph]
        
        
        if loc[0] == 3 and grid[2][loc[1]] != '.':
            return []
        
        if loc[1] == home:
            if loc[0] ==3:
                return []
            
            if grid[3][home] == amph and grid[2][home] == amph:
                return []
        
        adjacent_states = []
        #leaving original position
        if loc[0] != 1 :
            leave_cost = cur_cost + (loc[0] - 1)*step_cost
            # look left
            l_idx = loc[1]-1
            while grid[1][l_idx] == '.':
                if l_idx in [3, 5, 7, 9]:
                    l_idx -=1
                else:
                    new_grid = self.point_swap(grid, loc, [1,l_idx])
                    new_cost = leave_cost + (loc[1] - l_idx)*step_cost
                    adjacent_states.append((self._create_str(new_grid), new_cost ))
                    l_idx -=1
            
            # look right
            r_idx = loc[1]+1
            while grid[1][r_idx] == '.':
                if r_idx in [3, 5, 7, 9]:
                    r_idx +=1
                else:
                    new_grid = self.point_swap(grid, loc, [1,r_idx])
                    new_cost = leave_cost + ( r_idx - loc[1] )*step_cost
                    adjacent_states.append((self._create_str(new_grid), new_cost ))
                    r_idx +=1
                
            return adjacent_states
        
        # Going Home 
        if loc[0] == 1 :
            if (    (grid[3][home] in [amph, '.']) 
                and (grid[2][home] in [amph, '.']) ):
                if loc[1] < home:
                    if min([ grid[1][loc[1]+i] == '.' for i in range(1, home-loc[1]+1 )]):
                        new_cost = cur_cost+  (home-loc[1])*step_cost
                        if grid[3][home] == '.':
                            new_cost += step_cost*2
                            new_grid = self.point_swap(grid,loc, [3,home])
                        else:
                            new_cost += step_cost
                            new_grid = self.point_swap(grid, loc, [2,home])
                        adjacent_states.append((self._create_str(new_grid), new_cost ))
                            
                if loc[1] > home:
                    if min([ grid[1][loc[1]-i] == '.' for i in range(1, loc[1]-home+1 )]):
                        new_cost = cur_cost+  (loc[1]-home)*step_cost
                        if grid[3][home] == '.':
                            new_cost += step_cost*2
                            new_grid = self.point_swap(grid, loc, [3,home])
                        else:
                            new_cost += step_cost
                            new_grid = self.point_swap(grid,loc, [2,home])
                        adjacent_states.append((self._create_str(new_grid), new_cost ))
                        
                return adjacent_states
                        
        return []
            
            
    def create_next_moves(self,):
        possible_states = []
        for amph, locs in self.amphi_locs.items():
            for pt in locs:
                if pt[1] == self.home_entry[amph]:
                    tot_rows = len(self.grid)
                    below_home = min([ self.grid[r][pt[1]] == amph for r in range(pt[0]+1, tot_rows)])
                    if below_home and self.grid[pt[0]-1][pt[1]] == '.':
                        continue
                possible_states.extend(self.move_amphipods(amph, pt))
        return possible_states
    
    def __lt__(self, other):
        return (self.cost_so_far, self._create_str(self.grid)) < (other.cost_so_far, self._create_str(other.grid))
        

In [36]:
import heapq
class PriorityQueue:
    def __init__(self):
        self.elements: list[tuple[float, AmphipodsState]] = []
    
    def empty(self) -> bool:
        return not self.elements
    
    def put(self, item: AmphipodsState, priority: float):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self) -> tuple:
        return heapq.heappop(self.elements)[1]

In [50]:
def added_cost(state):
    amphi_locs = state.amphi_locs.copy()
    home_entry = {
            'A' : 3,
            'B' : 5,
            'C' : 7,
            'D' : 9
        }
    dist_off = 0
    for amph, locs in amphi_locs.items():
        dist_off += sum([ abs(loc[0]-5)+abs(loc[1] - home_entry[amph]) for loc in locs])
    return dist_off*1000
    

In [97]:
def added_cost(state,target_str):
    cur_state = str(state)
    
    dist_off = 0
    for i in range(len(target_str)):
         dist_off += int(cur_state[i] != target_str[i])
    return dist_off*500
    

In [98]:
END = '#############\n#...........#\n###A#B#C#D###\n  #A#B#C#D#\n  #########'
def perform_the_sort(init_state, end=END):
    explore = PriorityQueue()
    explore.put(init_state,0)
    cost_so_far = {str(init_state): 0}
    
    while not explore.empty():
        cur_state = explore.get()
        if str(cur_state) == end:
            break
            
        for neighbor,cost in cur_state.create_next_moves():
           
            neighbor_state = AmphipodsState(state=neighbor, coming_from=deepcopy(cur_state), cost_so_far=cost )
            
            if neighbor not in cost_so_far.keys() or cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = cost 
                priority = cost + added_cost(neighbor_state, END)
                explore.put(neighbor_state, priority)
                
    return cur_state
       

In [99]:
with open('data/test_23.txt', 'r') as f:
    test = ''.join(f.readlines())

In [100]:
test_state = AmphipodsState(state=test)

In [101]:
%%time
end_state = perform_the_sort(test_state)
print(end_state)
end_state.cost_so_far

#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
CPU times: user 9.75 s, sys: 40.3 ms, total: 9.79 s
Wall time: 9.8 s


12521

In [155]:
cur_state = end_state
while cur_state.coming_from is not None:
    print(str(cur_state) +f'\t{cur_state.cost_so_far}')
    cur_state = cur_state.coming_from

#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########	12521
#############
#.....D.....#
###A#B#C#.###
  #A#B#C#D#
  #########	8521
#############
#.....D...D.#
###A#B#C#.###
  #A#B#C#.#
  #########	5521
#############
#...B.D...D.#
###A#.#C#.###
  #A#B#C#.#
  #########	5501
#############
#.A.B.D...D.#
###.#.#C#.###
  #A#B#C#.#
  #########	5499
#############
#.A...D...D.#
###B#.#C#.###
  #A#B#C#.#
  #########	5479
#############
#.A.B.D...D.#
###B#.#C#.###
  #A#.#C#.#
  #########	5449
#############
#.A.B.....D.#
###B#.#C#.###
  #A#D#C#.#
  #########	2449
#############
#.A.B.C...D.#
###B#.#.#.###
  #A#D#C#.#
  #########	2249
#############
#.A.B.....D.#
###B#C#.#.###
  #A#D#C#.#
  #########	2049
#############
#.A.......D.#
###B#C#B#.###
  #A#D#C#.#
  #########	2009
#############
#.........D.#
###B#C#B#.###
  #A#D#C#A#
  #########	2000


In [25]:
with open('data/input_23.txt', 'r') as f:
    prod = ''.join(f.readlines())

In [16]:
prod_state = AmphipodsState(state=prod)

In [160]:
%%time
end_state = perform_the_sort(prod_state)
print(end_state)
end_state.cost_so_far

#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
CPU times: user 1min 34s, sys: 252 ms, total: 1min 35s
Wall time: 1min 35s


18195

# Part 2
- 2 more layers check

In [102]:
from copy import deepcopy
class Amphipods4layers(AmphipodsState):
    def __init__(self, state,coming_from=None, cost_so_far=0, add_lines=False):
        AmphipodsState.__init__(self,state,coming_from=coming_from, cost_so_far=cost_so_far)
        self.grid, self.amphi_locs = self.read_state(state, add_lines)
    
    def read_state(self,state, add_lines = False):
        extra_lines=[
                '  #D#C#B#A#',
                '  #D#B#A#C#',
                ]
        
        lines = state.split('\n')
        grid= []
        grid.append([ i for i in lines[0] ])
        amphi_locs = {'A':[],
                      'B':[],
                      'C':[],
                      'D':[],
                     }
        if add_lines:
            lines = lines[:3]+extra_lines+lines[3:]
        
        for r,line in enumerate(lines[1:]):
            new_row = []
            for c,element in enumerate(line):
                if element in  ['#', ' ', '.']:
                    new_row.append(element)
                    continue
                amphi_locs[element].append((r+1, c))
                new_row.append(element)
            grid.append(new_row)
        return grid, amphi_locs
    
    def move_amphipods(self,amph, loc):
        home = self.home_entry[amph]
        grid = self.grid.copy()
        cur_cost = self.cost_so_far
        step_cost = self.type_cost[amph]
        
        
        if loc[0] > 2 and grid[loc[0]-1][loc[1]] != '.':
            return []
        
        if loc[1] == home:
            if loc[0] ==5:
                return []
            
            if (grid[5][home] == amph 
                and grid[4][home] == amph
                and grid[3][home] == amph
                and grid[2][home] == amph):
                return []
        
        adjacent_states = []
        #leaving original position
        if loc[0] != 1 :
            leave_cost = cur_cost + (loc[0] - 1)*step_cost
            # look left
            l_idx = loc[1]-1
            while grid[1][l_idx] == '.':
                if l_idx in [3, 5, 7, 9]:
                    l_idx -=1
                else:
                    new_grid = self.point_swap(grid, loc, [1,l_idx])
                    new_cost = leave_cost + (loc[1] - l_idx)*step_cost
                    adjacent_states.append((self._create_str(new_grid), new_cost ))
                    l_idx -=1
            
            # look right
            r_idx = loc[1]+1
            while grid[1][r_idx] == '.':
                if r_idx in [3, 5, 7, 9]:
                    r_idx +=1
                else:
                    new_grid = self.point_swap(grid, loc, [1,r_idx])
                    new_cost = leave_cost + ( r_idx - loc[1] )*step_cost
                    adjacent_states.append((self._create_str(new_grid), new_cost ))
                    r_idx +=1
                
            return adjacent_states
        
        # Going Home 
        if loc[0] == 1 :
            if (    (grid[2][home] in [amph, '.']) 
                and (grid[3][home] in [amph, '.'])
                and (grid[4][home] in [amph, '.'])
                and (grid[5][home] in [amph, '.'])
               ):
                if loc[1] < home:
                    if min([ grid[1][loc[1]+i] == '.' for i in range(1, home-loc[1]+1 )]):
                        new_cost = cur_cost+  (home-loc[1])*step_cost
                        row=2
                        while grid[row][home] == '.':
                            row+=1
                        row -=1
                        new_cost += step_cost*(row-1)
                        new_grid = self.point_swap(grid,loc, [row,home])
                        adjacent_states.append((self._create_str(new_grid), new_cost ))
                            
                if loc[1] > home:
                    if min([ grid[1][loc[1]-i] == '.' for i in range(1, loc[1]-home+1 )]):
                        new_cost = cur_cost+  (loc[1]-home)*step_cost
                        row=2
                        while grid[row][home] == '.':
                            row+=1
                        row -=1
                        new_cost += step_cost*(row-1)
                        new_grid = self.point_swap(grid,loc, [row,home])
                        adjacent_states.append((self._create_str(new_grid), new_cost ))
                        
                return adjacent_states
        return []

In [109]:
PT2_END = '#############\n#...........#\n###A#B#C#D###\n  #A#B#C#D#\n  #A#B#C#D#\n  #A#B#C#D#\n  #########'
def perform_the_sort_part2(init_state, end = PT2_END):
    explore = PriorityQueue()
    explore.put(init_state,0)
    cost_so_far = {str(init_state): 0}
    
    while not explore.empty():
        cur_state = explore.get()
        if str(cur_state) == end:
            break
            
        for neighbor,cost in cur_state.create_next_moves():
           
            neighbor_state = Amphipods4layers(state=neighbor, coming_from=deepcopy(cur_state), cost_so_far=cost)
            
            if neighbor not in cost_so_far.keys() or cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = cost 
                priority = cost# + added_cost(neighbor_state, PT2_END)
                explore.put(neighbor_state, priority)
                

    return cur_state
       

In [110]:
with open('data/test_23_1.txt', 'r') as f:
    test_two = ''.join(f.readlines())

In [111]:
test_state = Amphipods4layers(state=test,add_lines=True)

In [112]:
%%time
end_state = perform_the_sort_part2(test_state,PT2_END)
print(end_state)
end_state.cost_so_far

#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #A#B#C#D#
  #A#B#C#D#
  #########
CPU times: user 3min 14s, sys: 624 ms, total: 3min 14s
Wall time: 3min 15s


44169

In [113]:
prod_state_pt2 = Amphipods4layers(state=prod, add_lines=True)

In [114]:
%%time
end_state = perform_the_sort_part2(prod_state_pt2)
print(end_state)
end_state.cost_so_far

#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #A#B#C#D#
  #A#B#C#D#
  #########
CPU times: user 1min 59s, sys: 652 ms, total: 1min 59s
Wall time: 2min


50265